In [1]:
#!pip install -q SentencePiece transformers
import torch
from transformers import T5EncoderModel, T5Tokenizer
import re
import os
import requests
from tqdm.auto import tqdm
import tqdm
from Bio import SeqIO
import gc
import pandas as pd
import numpy as np

tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False, cache_dir="/home/ubuntu/data/hai/huggingface_cache" )
model = T5EncoderModel.from_pretrained("Rostlab/prot_t5_xl_uniref50", cache_dir="/home/ubuntu/data/hai/huggingface_cache")
# model = model.half()
gc.collect()

# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model = model.to(device)
model = model.eval()


/home/ubuntu/data/miniconda3/envs/finetune-dephos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/data/miniconda3/envs/finetune-dephos/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as 

In [2]:
seq = "MDPSGVKVLETAEDIQERRQQVLDRYHRFKELSTLRRQKLEDSYRFQFFQRDAEELEKWIQEKLQIASDENYKDPTNLQGKLQKHQAFEAEVQANSGAIVKLDETGNLMISEGHFASETIRTRLMELHRQWELLLEKMREKGIKLLQAQKLVQYLRECEDVMDWINDKEAIVTSEELGQDLEHVEVLQKKFEEFQTDMAAHEERVNEVNQFAAKLIQEQHPEEELIKTKQDEVNAAWQRLKGLALQRQGKLFGAAEVQRFNRDVDETISWIKEKEQLMASDDFGRDLASVQALLRKHEGLERDLAALEDKVKALCAEADRLQQSHPLSATQIQVKREELITNWEQIRTLAAERHARLNDSYRLQRFLADFRDLTSWVTEMKALINADELASDVAGAEALLDRHQEHKGEIDAHEDSFKSADESGQALLAAGHYASDEVREKLTVLSEERAALLELWELRRQQYEQCMDLQLFYRDTEQVDNWMSKQEAFLLNEDLGDSLDSVEALLKKHEDFEKSLSAQEEKITALDEFATKLIQNNHYAMEDVATRRDALLSRRNALHERAMRRRAQLADSFHLQQFFRDSDELKSWVNEKMKTATDEAYKDPSNLQGKVQKHQAFEAELSANQSRIDALEKAGQKLIDVNHYAKDEVAARMNEVISLWKKLLEATELKGIKLREANQQQQFNRNVEDIELWLYEVEGHLASDDYGKDLTNVQNLQKKHALLEADVAAHQDRIDGITIQARQFQDAGHFDAENIKKKQEALVARYEALKEPMVARKQKLADSLRLQQLFRDVEDEETWIREKEPIAASTNRGKDLIGVQNLLKKHQALQAEIAGHEPRIKAVTQKGNAMVEEGHFAAEDVKAKLHELNQKWEALKAKASQRRQDLEDSLQAQQYFADANEAESWMREKEPIVGSTDYGKDEDSAEALLKKHEALMSDLSAYGSSIQALREQAQSCRQQVAPTDDETGKELVLALYDYQEKSPREVTMKKGDILTLLNSTNKDWWKVEVNDRQGFVPAAYVKKLDPAQSASRENLLEEQGSIALRQEQIDNQTRITKEAGSVSLRMKQVEELYHSLLELGEKRKGMLEKSCKKFMLFREANELQQWINEKEAALTSEEVGADLEQVEVLQKKFDDFQKDLKANESRLKDINKVAEDLESEGLMAEEVQAVQQQEVYGMMPRDETDSKTASPWKSARLMVHTVATFNSIKELNERWRSLQQLAEERSQLLGSAHEVQRFHRDADETKEWIEEKNQALNTDNYGHDLASVQALQRKHEGFERDLAALGDKVNSLGETAERLIQSHPESAEDLQEKCTELNQAWSSLGKRADQRKAKLGDSHDLQRFLSDFRDLMSWINGIRGLVSSDELAKDVTGAEALLERHQEHRTEIDARAGTFQAFEQFGQQLLAHGHYASPEIKQKLDILDQERADLEKAWVQRRMMLDQCLELQLFHRDCEQAENWMAAREAFLNTEDKGDSLDSVEALIKKHEDFDKAINVQEEKIAALQAFADQLIAAGHYAKGDISSRRNEVLDRWRRLKAQMIEKRSKLGESQTLQQFSRDVDEIEAWISEKLQTASDESYKDPTNIQSKHQKHQAFEAELHANADRIRGVIDMGNSLIERGACAGSEDAVKARLAALADQWQFLVQKSAEKSQKLKEANKQQNFNTGIKDFDFWLSEVEALLASEDYGKDLASVNNLLKKHQLLEADISAHEDRLKDLNSQADSLMTSSAFDTSQVKDKRDTINGRFQKIKSMAASRRAKLNESHRLHQFFRDMDDEESWIKEKKLLVGSEDYGRDLTGVQNLRKKHKRLEAELAAHEPAIQGVLDTGKKLSDDNTIGKEEIQQRLAQFVEHWKELKQLAAARGQRLEESLEYQQFVANVEEEEAWINEKMTLVASEDYGDTLAAIQGLLKKHEAFETDFTVHKDRVNDVCTNGQDLIKKNNHHEENISSKMKGLNGKVSDLEKAAAQRKAKLDENSAFLQFNWKADVVESWIGEKENSLKTDDYGRDLSSVQTLLTKQETFDAGLQAFQQEGIANITALKDQLLAAKHVQSKAIEARHASLMKRWSQLLANSAARKKKLLEAQSHFRKVEDLFLTFAKKASAFNSWFENAEEDLTDPVRCNSLEEIKALREAHDAFRSSLSSAQADFNQLAELDRQIKSFRVASNPYTWFTMEALEETWRNLQKIIKERELELQKEQRRQEENDKLRQEFAQHANAFHQWIQETRTYLLDGSCMVEESGTLESQLEATKRKHQEIRAMRSQLKKIEDLGAAMEEALILDNKYTEHSTVGLAQQWDQLDQLGMRMQHNLEQQIQARNTTGVTEEALKEFSMMFKHFDKDKSGRLNHQEFKSCLRSLGYDLPMVEEGEPDPEFEAILDTVDPNRDGHVSLQEYMAFMISRETENVKSSEEIESAFRALSSEGKPYVTKEELYQNLTREQADYCVSHMKPYVDGKGRELPTAFDYVEFTRSLFVN"

seq_with_space = " ".join(list(seq))
seq_with_space

'M D P S G V K V L E T A E D I Q E R R Q Q V L D R Y H R F K E L S T L R R Q K L E D S Y R F Q F F Q R D A E E L E K W I Q E K L Q I A S D E N Y K D P T N L Q G K L Q K H Q A F E A E V Q A N S G A I V K L D E T G N L M I S E G H F A S E T I R T R L M E L H R Q W E L L L E K M R E K G I K L L Q A Q K L V Q Y L R E C E D V M D W I N D K E A I V T S E E L G Q D L E H V E V L Q K K F E E F Q T D M A A H E E R V N E V N Q F A A K L I Q E Q H P E E E L I K T K Q D E V N A A W Q R L K G L A L Q R Q G K L F G A A E V Q R F N R D V D E T I S W I K E K E Q L M A S D D F G R D L A S V Q A L L R K H E G L E R D L A A L E D K V K A L C A E A D R L Q Q S H P L S A T Q I Q V K R E E L I T N W E Q I R T L A A E R H A R L N D S Y R L Q R F L A D F R D L T S W V T E M K A L I N A D E L A S D V A G A E A L L D R H Q E H K G E I D A H E D S F K S A D E S G Q A L L A A G H Y A S D E V R E K L T V L S E E R A A L L E L W E L R R Q Q Y E Q C M D L Q L F Y R D T E Q V D N W M S K Q E A F L L N E D L G D S L D

In [3]:
sequences_Example = [seq_with_space]
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True)
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

with torch.no_grad():
    embedding = model(input_ids=input_ids,attention_mask=attention_mask)
    
embedding = embedding.last_hidden_state.cpu().numpy()
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][:seq_len-1]
    features.append(seq_emd)
print(features)


[array([[-0.16561644, -0.28367668,  0.04488695, ...,  0.01789412,
        -0.05210443,  0.42441863],
       [-0.11601214, -0.14123383,  0.12363178, ..., -0.02427511,
         0.0724756 ,  0.31164733],
       [ 0.03362304, -0.14967978, -0.00378486, ...,  0.08800733,
        -0.05638671,  0.1320963 ],
       ...,
       [ 0.04602677, -0.03161727,  0.15122224, ..., -0.16175139,
        -0.01827634,  0.18742678],
       [-0.0394611 , -0.05286238,  0.23083697, ..., -0.06039274,
        -0.24827601, -0.06116877],
       [-0.10292251, -0.16868016,  0.11236537, ..., -0.20258453,
        -0.13289404, -0.03754392]], dtype=float32)]


In [6]:
features[0][980]

array([-0.36163762, -0.14155905,  0.08710913, ..., -0.09557725,
        0.21063912,  0.23064719], dtype=float32)

In [48]:
# Make features[0][226] to a string to copy
arr = features[0][226].tolist()

In [52]:
str = ','.join(map(lambda x: f'{x:.8f}', arr))

In [53]:
str.count(',')

1023

In [7]:
print(','.join(map(lambda x: f'{x:.8f}', features[0][1658])))

0.09429359,-0.01066891,0.11918972,-0.04070876,-0.27567831,0.10949013,0.08359051,-0.11480361,0.26767522,-0.39993453,-0.08736082,-0.02495143,-0.05796880,0.01967206,-0.30189613,0.39139587,0.24655095,-0.04347818,-0.19240431,0.07078072,0.01599681,0.00552984,0.01959108,0.30898681,-0.03875392,0.13704629,-0.08013632,-0.06982169,0.50032455,0.26233080,-0.20520982,-0.08523665,-0.10860712,-0.37017837,-0.13178942,0.43245122,0.32751796,-0.03085188,-0.04280655,-0.07750595,0.08652882,0.03099537,-0.23184557,-0.26624268,0.05032551,0.03415702,-0.04045527,-0.16747266,-0.15517090,0.10767906,0.38351578,-0.31747797,-0.08881777,0.26475471,0.04062501,0.13119912,-0.00695928,-0.19378619,0.29309773,0.07954207,0.22396305,-0.10136850,0.13347891,-0.01219326,-0.23096602,-0.07577585,0.16527443,-0.02699243,0.10029404,-0.02706888,0.04740941,-0.15591308,-0.21673825,-0.35657862,0.02952411,0.19523284,-0.17241968,0.05640265,-0.08146016,0.12948138,0.07541046,0.13395078,0.03893119,-0.20634192,-0.17242126,-0.18527250,-0.179296